In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [18]:
spark = SparkSession.builder.appName("best_book").getOrCreate()

In [19]:
dataPath = "hdfs://localhost:9000/user/nhom7/book/data/"
ratingsFilePath = dataPath + "BX-Book-Ratings.csv"
rating_df = spark.read.options(inferSchema="true", header="true", delimiter=';').csv(ratingsFilePath)

In [20]:
user_read_count = rating_df.groupBy('User-ID') \
                            .agg(F.count(rating_df['Book-Rating']).alias('Read-Count')) \
                    
user_read_count = user_read_count.filter(user_read_count['Read-Count'] > 10) \
                                    .orderBy('Read-Count', ascending = True)

user_read_count.show(10)

+-------+----------+
|User-ID|Read-Count|
+-------+----------+
|  51071|        11|
|   5452|        11|
|  77790|        11|
|  42430|        11|
|  64983|        11|
|  63574|        11|
|  26518|        11|
|  58878|        11|
|  28953|        11|
|  26823|        11|
+-------+----------+
only showing top 10 rows



In [21]:
improved_rating = rating_df.join(user_read_count, 'User-ID').select('ISBN', 'Book-Rating')

In [23]:
book_read_count = improved_rating.groupBy('ISBN') \
                            .agg(F.count(improved_rating['Book-Rating']).alias('Read-Count'), 
                                F.avg(improved_rating['Book-Rating']).alias('Average')) \

best_book = book_read_count.filter(book_read_count['Read-Count'] > 50) \
                            .orderBy('Average', ascending = False)

best_book.show(20)

+----------+----------+-----------------+
|      ISBN|Read-Count|          Average|
+----------+----------+-----------------+
|0439064864|       148| 6.54054054054054|
|0439136350|       158|6.531645569620253|
|0439139597|       152|              6.5|
|0590353403|       131|6.282442748091603|
|0877017883|        62|6.129032258064516|
|0618002227|        54|5.851851851851852|
|0156528207|        59|5.491525423728813|
|043935806X|       267|5.479400749063671|
|0552131067|        64|         5.109375|
|0439136369|       178|4.949438202247191|
|0812550706|       135|4.911111111111111|
|0553296981|       117|4.880341880341881|
|0439139600|       159|4.861635220125786|
|1400032717|        81|4.802469135802469|
|0618002219|        62|4.790322580645161|
|0452279178|        52|             4.75|
|0064471047|        63|4.746031746031746|
|0684853515|        64|          4.71875|
|038548951X|        53| 4.69811320754717|
|0552137030|        79|4.670886075949367|
+----------+----------+-----------